한국어 영화 리뷰 - BERT

In [1]:
import pandas as pd
url = "https://drive.google.com/uc?id=1KOKgZ4qCg49bgj1QNTwk1Vd29soeB27o"
df = pd.read_csv(url)

In [2]:
len(df)

14725

In [3]:
# rating 6이상이면 긍정  라벨 생성  y로 저장
import numpy as np
from sklearn.model_selection import train_test_split
# y를 생성해서 review컬럼이 x  데이터 분할  데이터 개수 확인
y = np.array([ 1 if value >=6 else 0 for value in  df.rating])
X = df.review.values
# 데이터셋을 학습 검증 평가 로 나눈다 x_train x_val x_test
X_,x_test,y_,y_test =  train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)
x_train,x_val,y_train,y_val =  train_test_split(X_,y_, test_size=0.2, random_state=42,stratify=y_)
print(x_train.shape,x_val.shape,x_test.shape)
print(y_train.shape,y_val.shape,y_test.shape)

(9424,) (2356,) (2945,)
(9424,) (2356,) (2945,)


In [4]:
%pip install evaluate

Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
# from datasets import load_metric  #  2022 이후로 HuggingFace에서 deprecated   evaluate
# old system -> legacy system
import evaluate
metric = evaluate.load('accuracy')


In [6]:
def compute_metrics(eval_pred):
  '''
  Args:
    eval_pred : logits,labels를 가지고있는 dataset
  Returns:
    accuracy
  '''
  logits, labels = eval_pred
  predictions =  np.argmax(logits,axis=-1)
  return metric.compute(predictions=predictions, references=labels)


In [7]:
# 데이터셋 생성  클래스(상속)
# __init__  __getitem__ __len__
# x,y        각각텐서    y의 개수
class OurDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels
  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx].clone().detach()) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item
  def __len__(self):
    return len(self.labels)



In [8]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')  # 다국어 지원(한국어)
print(tokenizer.tokenize("안녕하세요. 반갑습니다."))

['안', '##녕', '##하', '##세', '##요', '.', '반', '##갑', '##습', '##니다', '.']


In [9]:
inputs = tokenizer("안녕하세요. 반갑습니다.")
inputs

{'input_ids': [101, 9521, 118741, 35506, 24982, 48549, 119, 9321, 118610, 119081, 48345, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
# 2022년 이후로는 Auto~~~~ 토크나이져와 모델을 사용하도록 권장(벤더사에서 업데이트시 유리)
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
print(tokenizer.tokenize("안녕하세요. 반갑습니다."))
inputs = tokenizer("안녕하세요. 반갑습니다.")
print(inputs)

['안', '##녕', '##하', '##세', '##요', '.', '반', '##갑', '##습', '##니다', '.']
{'input_ids': [101, 9521, 118741, 35506, 24982, 48549, 119, 9321, 118610, 119081, 48345, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [11]:
# mBERT + Trainer로 미세조정(Fine Turning)
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
# 토큰화
train_input = tokenizer(x_train.tolist(), truncation=True, padding=True,return_tensors='pt')
val_input = tokenizer(x_val.tolist(), truncation=True, padding=True,return_tensors='pt')
test_input = tokenizer(x_test.tolist(), truncation=True, padding=True,return_tensors='pt')

# Dataset생성
train_dataset = OurDataset(train_input,y_train)
val_dataset = OurDataset(val_input,y_val)
test_dataset = OurDataset(test_input,y_test)
# 분류모델 생성
model = AutoModelForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
# Trainer에 사용할 Argument 셋팅
training_args = TrainingArguments(
    output_dir = './results',
    num_train_epochs = 1,
    eval_strategy = 'steps',
    eval_steps = 500,
    report_to = 'none'
)
# Trainer객체생성 및 학습(미세조정)
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    compute_metrics = compute_metrics
)
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\khh11\AppData\Local\Temp\ipykernel_21756\2354080707.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx].clone().detach()) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to c:\users\khh11\appdata\local\temp\pip-req-build-7r4ueiim
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit fcd729f2f4b37858f333597c0782388ada51eb5f
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' 'C:\Users\khh11\AppData\Local\Temp\pip-req-build-7r4ueiim'
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [48 lines of output]
      c:\Users\khh11\miniconda3\envs\pyt_env\Lib\site-packages\setuptools\__init__.py:92: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
      
              This deprecation is overdue, please update your project and remove deprecated
              calls to avoid build errors in the future.
              ********************************************************************************
      
      !!
     

KoBERT

In [13]:
%pip install sentencepiece

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 12.6 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer\&subdirectory=kobert_hf

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer": Expected package name at the start of dependency specifier
    'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer
    ^
Hint: = is not a valid operator. Did you mean == ?
'subdirectory'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [5]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")

ModuleNotFoundError: No module named 'kobert_tokenizer'

In [7]:
import torch
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1')
text = "한국어 모델을 공유합니다."
inputs = tokenizer.batch_encode_plus([text])
out = model(input_ids = torch.tensor(inputs['input_ids']),
              attention_mask = torch.tensor(inputs['attention_mask']))
out.pooler_output.shape
torch.Size([1, 768])


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

c:\Users\khh11\miniconda3\envs\pyt_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\khh11\.cache\huggingface\hub\models--skt--kobert-base-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

NameError: name 'tokenizer' is not defined